In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# 参数设置（二维归一化单位）
GM = 4 * np.pi**2      # 引力常数×太阳质量 (AU³/year²)
years = 5             # 模拟时间（年）
dt = 0.0005             # 输出时间步长（年）

# 初始条件（二维近日点数据）
y0 = [0.9833, 0.0,     # 位置 (AU) - x轴方向为近日点
       0.0, 6.387]     # 速度 (AU/year) - y轴方向速度

def derivatives(t, y):
    """ 二维轨道微分方程 """
    x, y_pos, vx, vy = y
    r = np.sqrt(x**2 + y_pos**2)
    ax = -GM * x / r**3
    ay = -GM * y_pos / r**3
    return [vx, vy, ax, ay]

# 积分时间设置
t_span = (0, years)
t_eval = np.arange(0, years, dt)

# 使用高精度RK45方法积分
sol = solve_ivp(derivatives, t_span, y0, 
                t_eval=t_eval, 
                method='RK45', 
                rtol=1e-9, 
                atol=1e-12)

# 提取结果
time = sol.t
x = sol.y[0]
y = sol.y[1]
vx = sol.y[2]
vy = sol.y[3]

# 计算远日点索引
r = np.sqrt(x**2 + y**2)
aphelion_idx = np.argmax(r)

# 计算附加参数
r = np.sqrt(x**2 + y**2)                      # 轨道半径
kinetic_energy = 0.5 * (vx**2 + vy**2)        # 动能
potential_energy = -GM / r                     # 势能
total_energy = kinetic_energy + potential_energy  # 总机械能
angular_momentum_z = x * vy - y * vx           #角动量

# 计算加速度
ax = np.zeros_like(time)
ay = np.zeros_like(time)
for i in range(len(time)):
    _, _, ax[i], ay[i] = derivatives(time[i], [x[i], y[i], vx[i], vy[i]])

# 创建DataFrame
df = pd.DataFrame({
    'time (year)': time,
    'x (AU)': x,
    'y (AU)': y,
    'vx (AU/year)': vx,
    'vy (AU/year)': vy,
    'ax (AU/year²)': ax,
    'ay (AU/year²)': ay,
    'orbital_radius (AU)': r,
    'kinetic_energy (AU²/year²)': kinetic_energy,
    'potential_energy (AU²/year²)': potential_energy,
    'total_energy (AU²/year²)': total_energy,
    'angular_momentum_z':  angular_momentum_z 
})

# 保存为CSV
file_path = "earth_orbit_simulation.csv"
df.to_csv(file_path, index=False)

# 验证输出
print(f"数据已保存至: {file_path}")
print(f"数据维度: {df.shape}")
print(df[['time (year)', 'x (AU)', 'y (AU)', 'orbital_radius (AU)']].head(3))
print("\n...\n")
print(df[['time (year)', 'x (AU)', 'y (AU)', 'orbital_radius (AU)']].tail(3))

# 计算远日点索引
r = np.sqrt(x**2 + y**2)
aphelion_idx = np.argmax(r)

# 绘制轨道
plt.figure(figsize=(8,8))
plt.plot(x, y, label='Earth Orbit', linewidth=0.8)
plt.plot(0, 0, 'yo', markersize=12, label='Sun')
plt.scatter(x[0], y[0], c='red', s=50, label='Perihelion')   # 近日点
plt.scatter(x[aphelion_idx], y[aphelion_idx], c='blue', s=50, label='Aphelion')  # 远日点

# 添加速度矢量箭头（初始时刻）
plt.xlabel('X (AU)')
plt.ylabel('Y (AU)')
plt.title(f'Earth Orbit Simulation ({years} Years)\nRK45 with rtol=1e-9')
plt.axis('equal')
plt.legend()
plt.grid(True)
plt.show()

# 轨道参数验证
print("[轨道参数验证]")
print(f"近日点距离: {r[0]:.4f} AU (理论值 0.9833 AU)")
print(f"远日点距离: {r[aphelion_idx]:.4f} AU (理论值 1.0167 AU)")
print(f"轨道偏心率: {(np.max(r) - np.min(r))/(np.max(r) + np.min(r)):.4f} (理论值 0.0167)")
print(f"总能量守恒误差: {np.std(0.5*(vx**2 + vy**2) - GM/r):.2e} AU²/year²")
print(f"相对波动率: {(np.std(angular_momentum_z)/angular_momentum_z[0])*1e6:.2f} ppm")